In [478]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

%matplotlib inline

In [479]:
###### 160906_band2 ########
# START_FRAME = 2
# END_FRAME = 5517
# SEQ_NAME = "160906_band2_with_ground"
# CONFIG_FILE = (
#     "/home/lei/rajrup/KinectStream/Multiview/config/panoptic_160906_band2.json"
# )

####### 170307_dance5 ########
# START_FRAME = 34
# END_FRAME = 10000
# SEQ_NAME = "170307_dance5_with_ground"
# CONFIG_FILE = (
#     "/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170307_dance5.json"
# )

####### 170915_office1 ########
# START_FRAME = 19
# END_FRAME = 5600
# SEQ_NAME = "170915_office1_with_ground"
# CONFIG_FILE = (
#     "/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170915_office1.json"
# )

####### 160906_pizza1 ########
# START_FRAME = 1
# END_FRAME = 1413
# SEQ_NAME = "160906_pizza1_with_ground"
# CONFIG_FILE = (
#     "/home/lei/rajrup/KinectStream/Multiview/config/panoptic_160906_pizza1.json"
# )

####### 170915_toddler4 ########
START_FRAME = 1
END_FRAME = 3800
SEQ_NAME = "170915_toddler4_with_ground"
CONFIG_FILE = (
    "/home/lei/rajrup/KinectStream/Multiview/config/panoptic_170915_toddler4.json"
)

with open(CONFIG_FILE) as f:
    config_data = json.load(f)
LOG_ID = config_data["log_id"]

In [480]:
# mm_type = "tracep1-scaled10.0"
mm_type = "wifi-25-scaled15.0"

with_stalls = True
# with_stalls = False

In [481]:
def calc_avg_pssim(
    df, start_frame, end_frame, offset_start=200, offset_end=50, with_stalls=True
):
    df = df.copy()
    df = df[df["Frame"] <= end_frame - offset_end]
    df = df[df["Frame"] >= start_frame + offset_start]

    pssim_geo = df["PSSIM_GEO_AB"].values
    pssim_color = df["PSSIM_COLOR_AB"].values

    num_stalls = 0
    num_render_frames = 0
    for frame_id in range(start_frame + offset_start, end_frame - offset_end + 1):
        if frame_id % 10 != 0:
            continue

        if frame_id in df["Frame"].values:
            num_render_frames += 1
        else:
            num_stalls += 1

    if with_stalls:
        num_frames = num_render_frames + num_stalls
    else:
        num_frames = num_render_frames

    stall_percent = 100.0 * num_stalls / (num_render_frames + num_stalls)

    avg_pssim_geo = np.sum(pssim_geo) / num_frames
    avg_pssim_color = np.sum(pssim_color) / num_frames
    return (
        avg_pssim_geo,
        avg_pssim_color,
        num_stalls,
        num_render_frames,
        num_frames,
        stall_percent,
    )

In [482]:
# DRACO

pipeline_method = "draco"
culling = "s_pcull_c_cull"

pssim_file_path = f"/datassd/pipeline_cpp/client_tiled/e2e_quality/{pipeline_method}/{SEQ_NAME}/{mm_type}/comp_o3d_pointssim_{culling}/log{LOG_ID}/3D_pssim.csv"
draco_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
draco_pssim_df.fillna(0, inplace=True)

# draco_pssim_df

In [483]:
(
    avg_pssim_geo,
    avg_pssim_color,
    num_stalls,
    num_render_frames,
    num_frames,
    stall_percent,
) = calc_avg_pssim(draco_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(
    f"Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%, num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}"
)

Method:draco, avg_pssim_geo: 18.25298591549296, avg_pssim_color: 19.11374647887324, stall_percent:80.56338028169014%, num_stalls: 286, num_render_frames: 69, num_frames: 355


In [484]:
pipeline_method = "livo_nocull"
culling = "s_nocull_c_cull"

pssim_file_path = f"/datassd/pipeline_cpp/client_tiled/e2e_quality/{pipeline_method}/{SEQ_NAME}/{mm_type}/comp_o3d_pointssim_{culling}/log{LOG_ID}/3D_pssim.csv"
livo_nocull_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
livo_nocull_pssim_df.fillna(0, inplace=True)

# livo_nocull_pssim_df

In [485]:
(
    avg_pssim_geo,
    avg_pssim_color,
    num_stalls,
    num_render_frames,
    num_frames,
    stall_percent,
) = calc_avg_pssim(
    livo_nocull_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls
)
print(
    f"Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%, num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}"
)

Method:livo_nocull, avg_pssim_geo: 78.344338028169, avg_pssim_color: 79.42095774647888, stall_percent:7.042253521126761%, num_stalls: 25, num_render_frames: 330, num_frames: 355


In [486]:
pipeline_method = "livo"
culling = "s_kpcull10_c_cull"

pssim_file_path = f"/datassd/pipeline_cpp/client_tiled/e2e_quality/{pipeline_method}/{SEQ_NAME}/{mm_type}/comp_o3d_pointssim_{culling}/log{LOG_ID}/3D_pssim.csv"
livo_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
livo_pssim_df.fillna(0, inplace=True)

# livo_pssim_df

In [487]:
(
    avg_pssim_geo,
    avg_pssim_color,
    num_stalls,
    num_render_frames,
    num_frames,
    stall_percent,
) = calc_avg_pssim(livo_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls)
print(
    f"Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%,  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}"
)

Method:livo, avg_pssim_geo: 88.1187323943662, avg_pssim_color: 85.36515492957747, stall_percent:1.1267605633802817%,  num_stalls: 4, num_render_frames: 351, num_frames: 355


In [488]:
pipeline_method = "starline"
culling = "s_nocull_c_cull"

pssim_file_path = f"/datassd/pipeline_cpp/client_tiled/e2e_quality/{pipeline_method}/{SEQ_NAME}/{mm_type}/comp_o3d_pointssim_{culling}/log{LOG_ID}/3D_pssim.csv"
starline_only_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
starline_only_pssim_df.fillna(0, inplace=True)

# starline_only_pssim_df

In [489]:
(
    avg_pssim_geo,
    avg_pssim_color,
    num_stalls,
    num_render_frames,
    num_frames,
    stall_percent,
) = calc_avg_pssim(
    starline_only_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls
)
print(
    f"Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%,  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}"
)

Method:starline, avg_pssim_geo: 31.561352112676058, avg_pssim_color: 32.77352112676056, stall_percent:61.690140845070424%,  num_stalls: 219, num_render_frames: 136, num_frames: 355


In [490]:
pipeline_method = "meshreduce"
culling = "s_nocull_c_nocull"

pssim_file_path = f"/hdd/pipeline_meshreduce/e2e_quality/mesh_to_ptcl/{SEQ_NAME}/{mm_type}/comp_draco_pointssim_{culling}/3D_pssim.csv"
meshreduce_only_pssim_df = pd.read_csv(pssim_file_path)

# replace NaN with 0
meshreduce_only_pssim_df.fillna(0, inplace=True)

# meshreduce_only_pssim_df

In [491]:
(
    avg_pssim_geo,
    avg_pssim_color,
    num_stalls,
    num_render_frames,
    num_frames,
    stall_percent,
) = calc_avg_pssim(
    meshreduce_only_pssim_df, START_FRAME, END_FRAME, with_stalls=with_stalls
)
print(
    f"Method:{pipeline_method}, avg_pssim_geo: {avg_pssim_geo}, avg_pssim_color: {avg_pssim_color}, stall_percent:{stall_percent}%,  num_stalls: {num_stalls}, num_render_frames: {num_render_frames}, num_frames: {num_frames}"
)

Method:meshreduce, avg_pssim_geo: 54.183239436619715, avg_pssim_color: 68.02825352112676, stall_percent:19.43661971830986%,  num_stalls: 69, num_render_frames: 286, num_frames: 355
